In [1]:
import datetime as dt
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import glob
from tidegauge_functions import read_GPS_SONEL

%matplotlib inline

In [2]:
datum = 'dAB48_XXXXXXXXX_JPL14.neu'
dir_in = 'data/GPS'

In [3]:
pattern = os.path.join(dir_in, datum)
print(pattern)

data/GPS\dAB48_XXXXXXXXX_JPL14.neu


In [4]:
filenames = sorted(glob.glob(pattern))  # , key=alphanum_key)

In [5]:
for f, filepath in enumerate(filenames):
    df = read_GPS_SONEL(filepath)
    print(f'\n\n{filepath}')
    print(df.head())



data/GPS\dAB48_XXXXXXXXX_JPL14.neu
            North   East  Vertical  NorthSTD  EastSTD  VerticalSTD
Year                                                              
2005-10-06  314.5  262.3     -67.3       1.2      0.7          3.7
2005-10-07  313.4  266.7     -65.5       1.1      0.7          3.3
2005-10-08  313.9  262.7     -63.4       1.0      0.6          3.2
2005-10-09  315.5  265.1     -64.0       1.0      0.6          3.2
2005-10-10  316.3  265.4     -64.7       1.1      0.7          3.2


In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
def decompose(df, var, site, period):  
    
    decomposition = seasonal_decompose(df[var], freq = period)

    trend = decomposition.trend
    seasonal = decomposition.seasonal
    residual = decomposition.resid
    Amplitude = max(seasonal) - min(seasonal)
    
    decomposition.plot();
    df['denoised'] = df[var] - residual
    df['trend'] = trend
    print(f'\n\n{filepath}')
    print(df.head())
    print(f'{site} Seasonal Amplitude =', Amplitude)
    seasonal_amp = pd.DataFrame(list(zip(list({site}), list(Amplitude))),
                            colums = ['Site_Name', 'Seasonal_Amplitude'])
    seasonal_amp.to_csv('GPS_Seasonal_Amplitude_SONEL')

# Change Point Detection
def change_pt_detection(df, var, points):
    model = "l1"  
    algo = rpt.Dynp(model=model, min_size=3, jump=5).fit(points)
    my_bkps = algo.predict(n_bkps=10)

    
def change_pt_plot()
    rpt.show.display(points, my_bkps, figsize=(10, 6))
    plt.suptitle(f"{site}Change Point Detection")
    plt.show()

In [18]:
def test_calc_OLS(df, var, start, end):
    df = df[start:end]
    x, y = np.arange(len(df[var].dropna())), df[var].dropna()
    x = sm.add_constant(x)
    model = sm.OLS(y, x)
    res = model.fit()
    return res

def test_plot_OLS(df, res, res2, site, var, trend_array, trend_array2, point1, point2):
    df1 = df[(df.index <= point1)]
    df2 = df[(df.index >= point2)]
    fig, ax = plt.subplots(1, 1, figsize=(12,6));
    ax.plot(df1[var].dropna().index, df1[var].dropna().values, 
            label='trend', marker='.', linestyle='', color = 'darkgrey')
    ax.plot(df1[var].dropna().index, [res1.params.x1*i + res1.params.const for i in np.arange(len(df1[var].dropna()))],
           marker = '', linestyle = '-', color = 'blue')
    # ax[1].plot(df['Vertical'].dropna().index, res.resid.values);
    # ax[1].plot(df['Vertical'].dropna().index, np.abs(res.resid.values));
    # ax[1].hlines(0, 0, len(res.resid), color='k');
    # ax[1].set_title("Residuals");
    ax.plot(df2[var].dropna().index, df2[var].dropna().values, 
            label='trend', marker='.', linestyle='', color = 'darkgrey')
    ax.plot(df2[var].dropna().index, [res2.params.x1*i + res2.params.const for i in np.arange(len(df2[var].dropna()))],
           marker = '', linestyle = '-', color = 'red')
    # ax[1].plot(df['Vertical'].dropna().index, res.resid.values);
    # ax[1].plot(df['Vertical'].dropna().index, np.abs(res.resid.values));
    # ax[1].hlines(0, 0, len(res.resid), color='k');
    plt.axvline(dt.datetime(2013, 1, 5), color= 'red', linestyle='--')
    ax.set_title(f" Pre 2013 Earthquake Trend = {trend_array * 1:.2f} mm/yr, Post 2013 Earthquake Trend= {trend_array2 * 1:.2f} mm/yr", fontsize=18);
    # ax[1].set_title("Residuals");
    plt.suptitle("Port Alexander, Alaska, USA", fontsize=20)
    plt.xlabel('Date', fontsize=18)
    plt.ylabel('Vertical Land Motion (mm)', fontsize=18)
    plt.rc('xtick',labelsize=16)
    plt.rc('ytick',labelsize=16)
    plt.savefig(f'figs/test_GPS_OLS_split_{site}.png')
    
    

In [ ]:
for f, filepath in enumerate(filenames):
    print(f'\n\n{filepath}')
    print(f'f: {f}')
    df = read_GPS_SONEL(filepath)
    df = decompose(df, 'Vertical', filepath[-24:-20], 365)
    

In [8]:
def convert_trend_toyearly(df, res):
    
    period = df.index.year.value_counts().max()
    yearlytrend = res.params.x1 * period
    
    return yearlytrend

In [ ]:
df = read_GPS_SONEL(filepath)
df.plot()

In [ ]:
SONEL_trend_array = np.full(np.shape(filenames)[0], np.nan)
SONEL_trend_array2 = np.full(np.shape(filenames)[0], np.nan)
site_name_array = []


for f, filepath in enumerate(filenames):
    print(f'\n\n{filepath}')
    print(f'f: {f}')
    ## ToDo - extract and keep other important thing about each site from the file...Lat/Lon, name, etc.
    
    # Read in data
    df = read_GPS_SONEL(filepath)
    
    # Get trend using linear regression
    res1 = test_calc_OLS(df, 'Vertical', '2005-10-6', '2013-1-4')
    SONEL_trend_array[f] = convert_trend_toyearly(df, res1)
    
    res2 = test_calc_OLS(df, 'Vertical', '2013-1-5','2020-1-1')
    SONEL_trend_array2[f] = convert_trend_toyearly(df, res2)
    
    # Save site name
    site_name_array.append(filepath[-24:-20])
    
    print(f"Trend 1 = {SONEL_trend_array[f] * 1:.2f} mm/yr")
    print(res1.summary())
    print(f"Trend 2 = {SONEL_trend_array2[f] * 1:.2f} mm/yr")
    print(res2.summary())
   
    # Make Plot
    test_plot_OLS(df, res1, res2, filepath[-18:-14], 'Vertical', SONEL_trend_array[f], SONEL_trend_array2[f], '2013-1-5','2013-1-5')